<a href="https://colab.research.google.com/github/proap900/AI_LAB/blob/main/vaccum_cleaner_agent_lab1_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Required Libraries
import random
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from collections import deque
from IPython.display import display, clear_output

# Grid Configuration
GRID_SIZE = 10       # 10x10 environment
DIRTY = 1            # Cell with dirt
CLEAN = 0            # Cell is clean

# Seed for reproducibility (optional)
random.seed(42)


In [ ]:
#Environment
class Environment:
    def __init__(self):
        # Initialize random dirt in grid
        self.grid = [[random.choice([DIRTY, CLEAN]) for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
        # Agent's initial random position
        self.agent_pos = (random.randint(0, GRID_SIZE - 1), random.randint(0, GRID_SIZE - 1))

        # Performance tracking
        self.steps_taken = 0
        self.cells_cleaned = 0
        self.visited_cells = set()
        self.revisit_count = 0

    def get_state(self, x, y):
        return self.grid[y][x]

    def clean(self, x, y):
        if self.grid[y][x] == DIRTY:
            self.grid[y][x] = CLEAN
            self.cells_cleaned += 1

    def is_clean(self):
        # Return True if no dirt left
        return all(cell == CLEAN for row in self.grid for cell in row)

    def move_agent(self, x, y):
        if (x, y) in self.visited_cells:
            self.revisit_count += 1
        else:
            self.visited_cells.add((x, y))
        self.agent_pos = (x, y)
        self.steps_taken += 1

    def get_neighbors(self, x, y):
        # 8 directions around agent
        directions = [(-1, -1), (0, -1), (1, -1),
                      (-1, 0),           (1, 0),
                      (-1, 1),  (0, 1),  (1, 1)]
        neighbors = {}
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < GRID_SIZE and 0 <= ny < GRID_SIZE:
                neighbors[(nx, ny)] = self.grid[ny][nx]
        return neighbors


In [ ]:
#Agents
class ReflexAgent:
    def __init__(self, env):
        self.env = env

    def step(self):
        x, y = self.env.agent_pos

        # Clean current cell if dirty
        if self.env.get_state(x, y) == DIRTY:
            self.env.clean(x, y)
            return

        # Move to any neighboring dirty cell if found
        neighbors = self.env.get_neighbors(x, y)
        for (nx, ny), state in neighbors.items():
            if state == DIRTY:
                self.env.move_agent(nx, ny)
                return

        # Otherwise move randomly to any neighbor
        if neighbors:
            self.env.move_agent(*random.choice(list(neighbors.keys())))

class GoalBasedAgent:
    def __init__(self, env):
        self.env = env

    def bfs_nearest_dirt(self, start):
        # BFS to find nearest dirty cell
        queue = deque([start])
        visited = set([start])
        parent = {start: None}

        while queue:
            current = queue.popleft()
            if self.env.get_state(*current) == DIRTY:
                # Reconstruct path back to start
                path = []
                while current != start:
                    path.append(current)
                    current = parent[current]
                path.reverse()
                return path  # Return path to nearest dirt

            for neighbor in self.env.get_neighbors(*current):
                if neighbor not in visited:
                    visited.add(neighbor)
                    parent[neighbor] = current
                    queue.append(neighbor)
        return []  # No dirt found

    def step(self):
        x, y = self.env.agent_pos

        # Clean if dirty
        if self.env.get_state(x, y) == DIRTY:
            self.env.clean(x, y)
            return

        # Find nearest dirty cell path
        path = self.bfs_nearest_dirt((x, y))

        # Move one step toward nearest dirt if exists
        if path:
            next_step = path[0]
            self.env.move_agent(*next_step)
        else:
            # No dirt found: random neighbor move to explore
            neighbors = self.env.get_neighbors(x, y)
            if neighbors:
                self.env.move_agent(*random.choice(list(neighbors.keys())))

class UtilityAgent:
    def __init__(self, env):
        self.env = env
        self.visit_freq = [[0 for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]

    def bfs_toward_dirt(self, start):
        queue = deque([start])
        visited = set([start])
        parent = {start: None}

        while queue:
            current = queue.popleft()
            if self.env.get_state(*current) == DIRTY:
                # Reconstruct path to dirty cell
                path = []
                while current != start:
                    path.append(current)
                    current = parent[current]
                path.reverse()
                return path
            for neighbor in self.env.get_neighbors(*current):
                if neighbor not in visited:
                    visited.add(neighbor)
                    parent[neighbor] = current
                    queue.append(neighbor)
        return []

    def step(self):
        x, y = self.env.agent_pos
        self.visit_freq[y][x] += 1

        if self.env.get_state(x, y) == DIRTY:
            self.env.clean(x, y)
            return

        neighbors = self.env.get_neighbors(x, y)
        best_pos = None
        best_utility = -float('inf')

        for (nx, ny), state in neighbors.items():
            utility = 0
            if state == DIRTY:
                utility += 10
            elif (nx, ny) not in self.env.visited_cells:
                utility += 2  # reward exploring
            # penalize frequently visited cells
            utility -= self.visit_freq[ny][nx] * 0.5
            if utility > best_utility:
                best_utility = utility
                best_pos = (nx, ny)

        if best_pos and best_utility > 0:
            self.env.move_agent(*best_pos)
        else:
            # If all neighbors have bad utility, use BFS to head toward far dirt
            path = self.bfs_toward_dirt((x, y))
            if path:
                next_step = path[0]
                self.env.move_agent(*next_step)
            else:
                # fallback: move randomly
                if neighbors:
                    self.env.move_agent(*random.choice(list(neighbors.keys())))



In [ ]:
#visualization
def draw_environment(env):
    fig, ax = plt.subplots(figsize=(6, 6))
    mat = np.zeros((GRID_SIZE, GRID_SIZE, 3))

    for y in range(GRID_SIZE):
        for x in range(GRID_SIZE):
            if env.get_state(x, y) == DIRTY:
                mat[y][x] = [0.6, 0.4, 0.2]  # Brown dirt
            else:
                mat[y][x] = [1.0, 1.0, 1.0]  # White clean

    ax.imshow(mat, interpolation='nearest')

    # Draw agent as orange circle
    x, y = env.agent_pos
    ax.add_patch(plt.Circle((x, y), 0.3, color='orange'))

    # Highlight neighbors with blue square border
    for nx, ny in env.get_neighbors(x, y):
        ax.add_patch(plt.Rectangle((nx - 0.5, ny - 0.5), 1, 1,
                                   edgecolor='blue', fill=False, linewidth=2))

    ax.set_xticks(np.arange(-0.5, GRID_SIZE, 1))
    ax.set_yticks(np.arange(-0.5, GRID_SIZE, 1))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.grid(True, color='black')
    plt.show()

def run_gui(agent_type="Utility"):
    env = Environment()

    # Initialize agent
    if agent_type == "Reflex":
        agent = ReflexAgent(env)
    elif agent_type == "GoalBased":
        agent = GoalBasedAgent(env)
    else:
        agent = UtilityAgent(env)

    # Setup UI
    button = widgets.Button(description=f"Step {agent_type} Agent")
    output = widgets.Output(layout={'border': '1px solid black', 'padding': '10px'})

    def on_click(b):
        with output:
            clear_output(wait=True)
            if not env.is_clean():
                agent.step()
            draw_environment(env)
            print(f"🤖 Agent Type: {agent_type}")
            print(f"🚶 Steps Taken: {env.steps_taken}")
            print(f"🧹 Cells Cleaned: {env.cells_cleaned}")
            print(f"🔁 Revisited Cells: {env.revisit_count}")
            if env.is_clean():
                print("✅ Cleaning complete!")

    # Bind the click event
    button.on_click(on_click)

    # Display layout: button on top
    ui = widgets.VBox([button, output])
    display(ui)

    # Initial visualization
    draw_environment(env)


# To run simulation, change agent_type: "Reflex", "GoalBased", or "Utility"
run_gui(agent_type="Utility")


In [ ]:
#performance summary
def run_full_sim(agent_type):
    env = Environment()
    if agent_type == "Reflex":
        agent = ReflexAgent(env)
    elif agent_type == "GoalBased":
        agent = GoalBasedAgent(env)
    else:
        agent = UtilityAgent(env)

    # Run until clean or max steps to avoid infinite loops
    max_steps = 1000
    while not env.is_clean() and env.steps_taken < max_steps:
        agent.step()

    return {
        "Agent": agent_type,
        "Steps Taken": env.steps_taken,
        "Cells Cleaned": env.cells_cleaned,
        "Revisited Cells": env.revisit_count
    }

# Run for all agents
results = []
for a_type in ["Reflex", "GoalBased", "Utility"]:
    results.append(run_full_sim(a_type))

# Plot results
labels = [r["Agent"] for r in results]
steps = [r["Steps Taken"] for r in results]
cleaned = [r["Cells Cleaned"] for r in results]
revisited = [r["Revisited Cells"] for r in results]

x = np.arange(len(labels))
width = 0.25

fig, ax = plt.subplots(figsize=(10, 5))
rects1 = ax.bar(x - width, steps, width, label='Steps Taken')
rects2 = ax.bar(x, cleaned, width, label='Cells Cleaned')
rects3 = ax.bar(x + width, revisited, width, label='Revisited Cells')

ax.set_ylabel('Count')
ax.set_title('Vacuum Agent Performance Comparison')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

for rect in rects1 + rects2 + rects3:
    height = rect.get_height()
    ax.annotate('{}'.format(height),
                xy=(rect.get_x() + rect.get_width()/2, height),
                xytext=(0,3),  # 3 points vertical offset
                textcoords="offset points",
                ha='center', va='bottom')

plt.show()
